## Rythme Generation Using LZify

In [1]:
def IPMotif(text, dictionary):
    """Compute an associative dictionary (the motif dictionary)."""
    
    motif = ""
    result = []
    for c in text:
        motif = motif + c
        if motif in dictionary:
            # Increase count for existing motif
#            print '%s in dictionary' % motif
            dictionary[motif] += 1
        else:
            # Add motif to the dictionary.
            dictionary[motif] = 1
            motif = ""
#            print 'add %s to dictionary' % motif

    return dictionary


In [2]:
from music21 import *

chorales = corpus.search('bach', fileExtensions='xml')

In [3]:
dict1 = {}
for i in range(len(chorales)):

    bwv = chorales[i].parse()
    sop = bwv.parts['Soprano']
    
    if sop is None:
        continue

    text = ""
    for note in sop.flat.notes:
        
        dur = int(note.duration.quarterLength * 4)
        
        text += str(dur)
         
    dict1 = IPMotif(text, dict1)

print(dict1)


{'4': 1486, '44': 1157, '444': 936, '4444': 716, '44444': 541, '444444': 366, '8': 224, '4444444': 251, '44444444': 209, '44444448': 18, '48': 118, '88': 51, '84': 139, '444444444': 175, '884': 24, '444448': 79, '8844': 20, '448': 68, '888': 16, '1': 147, '6': 68, '88444': 14, '488': 10, '16': 17, '884444': 12, '8888': 9, '168': 3, '844': 88, '4484': 31, '4881': 1, '61': 5, '616': 2, '4444444444': 146, '444444442': 17, '2': 718, '44444444444': 125, '444444444444': 97, '22': 229, '44442': 77, '24': 330, '4444442': 33, '244': 215, '42': 138, '242': 57, '2444': 171, '4444444444444': 72, '224': 125, '444442': 57, '24444': 132, '2244': 101, '4444444442': 11, '28': 78, '4442': 101, '11': 27, '22444': 80, '4444448': 64, '422': 128, '444444444442': 15, '284': 61, '224444': 64, '4444422': 49, '2244444': 45, '44444444444444': 51, '44422': 92, '2242': 12, '44444224': 31, '4224': 82, '42244': 65, '442': 111, '2446': 3, '221': 6, '12': 61, '2844': 39, '4422': 104, '444422': 74, '44224': 66, '444444

In [4]:
def IPContinuation(dict1):
    """Compute continuation dictionary from a motif dictionary"""
    
    dict2 = {}
    for Wk in dict1:
        counter = dict1[Wk]
        W = Wk[:-1]
        #print 'W =',W
        k = Wk[-1]
        #print 'k-',k
        if W in dict2:
            dict2[W].append((k,counter))
            #print 'dict2', dict2
        else:
            dict2[W] = [(k,counter)]
            #print 'dict2', dict2
    dict2 = Normalize(dict2)    
    return dict2
                    
                    
def Normalize(dict2):
    """Turns the counters in every element of dict2 to probabilities"""
    
    for W in dict2:
        cnt = [tup[1] for tup in dict2[W]]
        ttl = sum(cnt)
        for k,tup in enumerate(dict2[W]):
            dict2[W][k] = (tup[0],float(tup[1])/ttl)
    return dict2

In [5]:
dict2 = IPContinuation(dict1)
print(dict2)

{'': [('4', 0.5596986817325801), ('8', 0.08436911487758945), ('1', 0.05536723163841808), ('6', 0.025612052730696798), ('2', 0.2704331450094162), ('3', 0.0007532956685499058), ('0', 0.003766478342749529)], '4': [('4', 0.7897610921501707), ('8', 0.08054607508532423), ('2', 0.09419795221843004), ('1', 0.015017064846416382), ('6', 0.01911262798634812), ('3', 0.0013651877133105802)], '44': [('4', 0.8174672489082969), ('8', 0.059388646288209605), ('2', 0.09694323144104804), ('1', 0.01572052401746725), ('6', 0.008733624454148471), ('3', 0.0017467248908296944)], '444': [('4', 0.7723840345199569), ('2', 0.10895361380798274), ('6', 0.017259978425026967), ('8', 0.07874865156418555), ('1', 0.019417475728155338), ('3', 0.003236245954692557)], '4444': [('4', 0.758765778401122), ('2', 0.10799438990182328), ('1', 0.023842917251051893), ('6', 0.016830294530154277), ('8', 0.091164095371669), ('3', 0.001402524544179523)], '44444': [('4', 0.6790352504638218), ('8', 0.14656771799628943), ('2', 0.1057513914

In [6]:
from numpy.random import multinomial as randm
from numpy import where

def IPGenerate(n,dict2):
    p = 0
    out = ""
    dur = 0
    while True:
        while True:
            context = out[-p:]
            if context in dict2:
                prob = [tup[1] for tup in dict2[context]]
                conti = where(randm(1,prob))[0][0]
                cont = dict2[context][conti][0]
                dur += int(cont)
                out = out + cont
#                print prob, conti, cont, out
                break
            else:
                p = p-1
                
        if dur > n:
            break
        
    return out

In [44]:
rythme = IPGenerate(200, dict2)

In [46]:
with open('rythme.txt', 'w') as f:
    f.write(text)
    
print(list(rythme))

['4', '8', '4', '8', '4', '1', '2', '8', '4', '8', '4', '8', '4', '1', '2', '2', '2', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '2', '2', '4', '4', '4', '4', '4', '4', '4', '1', '2', '4', '4', '4', '4', '4', '1', '2', '4', '4', '4', '4', '4', '8']


## Spatialization Generation Using Markov Chain


In [38]:
def Markov(text,order=0):
    """Compute a Markov models (fixed length motif dictionary)."""
    
    dict3 = {}
    for i in range(len(text)-order):
        W = text[i:i+order]
        #print 'W', W
        k  = text[i+order]
        #print 'k', k
        if W in dict3:
            if k in list(zip(*dict3[W]))[0]:
                dict3[W][k] += 1
                #print('dict', dict3)
            else:
                dict3[W][k] = 1
                #print('dict', dict3)
        else:
            dict3[W] = {k:1}
            #print('dict', dict3)

    for x in dict3:
        dict3[x] = list(dict3[x].items())
    dict3 = Normalize(dict3)    
    return dict3


In [39]:
from collections import Counter

pitch_lookup = {
    'C': 0,
    'D': 1,
    'E': 2,
    'F': 3,
    'G': 4,
    'A': 5,
    'B': 6,
}

pitch = ''
for i in range(len(chorales)):

    bwv = chorales[i].parse()
    sop = bwv.parts['Soprano']
    
    if sop is None:
        continue

    for note in sop.flat.notes:
        
        pic = str(note.pitch)
        
        if len(pic) > 2:
            pitch += str(7)
        else:
            pitch += str(pitch_lookup[pic[0]])

print(Counter(list(pitch)))

Counter({'7': 4220, '5': 3297, '1': 3076, '6': 2679, '0': 2374, '4': 2309, '2': 1879, '3': 1067})


In [40]:
def MaxOrder(text,order):
    dictall = {}
    for o in range(order):
        dicto = Markov(text,o)
        dictall.update(dicto)
    return dictall

In [41]:
dictM1 = MaxOrder(pitch,5)
print(dictM1)

{'': [('3', 0.05105018898617291), ('0', 0.11358308214917946), ('5', 0.1577436486292522), ('1', 0.14716999186641788), ('2', 0.08990000478446007), ('7', 0.2019042151093249), ('4', 0.11047318310128702), ('6', 0.12817568537390556)], '3': [('0', 0.05060918462980318), ('2', 0.274601686972821), ('3', 0.1471415182755389), ('1', 0.04967197750702906), ('4', 0.23711340206185566), ('5', 0.038425492033739454), ('7', 0.19962511715089035), ('6', 0.0028116213683223993)], '0': [('5', 0.06402695871946082), ('1', 0.2695871946082561), ('0', 0.15332771693344566), ('7', 0.17944397641112048), ('4', 0.03327716933445661), ('6', 0.2704296545914069), ('3', 0.018955349620893007), ('2', 0.010951979780960405)], '5': [('3', 0.022747952684258416), ('1', 0.05398847437063998), ('4', 0.29693660903851987), ('0', 0.022747952684258416), ('5', 0.1710646041856233), ('7', 0.16681831968456173), ('6', 0.24749772520473157), ('2', 0.018198362147406732)], '1': [('1', 0.1771781534460338), ('0', 0.24869960988296488), ('2', 0.1853055

In [47]:
spatial = IPGenerate(len(rythme), dictM1)
with open('spatial.txt', 'w') as f:
    f.write(spatial)
    
print(list(spatial))

['1', '5', '1', '4', '5', '7', '4', '3', '5', '6', '0', '5', '4', '4']
